# 1. [60+10] Language and Topic models

A common suggestion to users for coming up with good queries is to think of words that would likely appear in a relevant document, and to use those words as the query. The language modeling approach to IR directly models this idea: a document is a good match to a query if the document model is likely to generate the query, which will in turn happen if the document contains the query words often. 

You will score documents with respect to user query using language models and also get some experience with topic modelling.

## 1.0. [5] Loading data

We use the dataset we already used once - [this topic-modeling dataset](https://code.google.com/archive/p/topic-modeling-tool/downloads) ([or from github](https://github.com/IUCVLab/information-retrieval/blob/main/datasets/topic-modelling.zip)).

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip 'drive/MyDrive/topic-modelling.zip'

Archive:  drive/MyDrive/topic-modelling.zip
  inflating: testdata_braininjury_10000docs.txt  
  inflating: testdata_news_economy_2073docs.txt  
  inflating: testdata_news_fuel_845docs.txt  
  inflating: testdata_news_music_2084docs.txt  


In [ ]:
# TODO: read the dataset
import numpy as np
all_data = np.array([])

with open("testdata_braininjury_10000docs.txt", 'r') as f:
    all_data = np.concatenate((all_data, f.read().split('\n')))[:-1]
# all_data = np.ravel(all_data)[:-1]
with open("testdata_news_economy_2073docs.txt", 'r') as f:
    all_data = np.concatenate((all_data, f.read().split('\n')))[:-1]
# all_data = np.ravel(all_data)[:-1]
with open("testdata_news_fuel_845docs.txt", 'r') as f:
    all_data = np.concatenate((all_data, f.read().split('\n')))[:-1]
# all_data = np.ravel(all_data)[:-1]
with open("testdata_news_music_2084docs.txt", 'r') as f:
    all_data = np.concatenate((all_data, f.read().split('\n')))[:-1]
# all_data = np.ravel(all_data)[:-1]

print("# of documents", len(all_data))
assert len(all_data) == 15002

# of documents 15002


## 1.1. [55] Ranking Using Language Models
Our goal is to rank documents by $P(d|q)$, where the probability of a document is interpreted as the likelihood that it is relevant to the query. 

Using Bayes rule: $P(d|q) = \frac{P(q|d)P(d)}{P(q)}$

$P(q)$ is the same for all documents, and so can be ignored. The prior probability of a document $P(d)$ is often treated as uniform across all $d$'s and so it can also be ignored. What does it mean? 

It means that computing $P(q|d)$ for different documents we can compare how relevant are they to the query. How can we estimate $P(q|d)$?

$P(q|d)$ can be estimated as:

![](https://i.imgur.com/BEIMAC1.png)

where $M_d$ is the language model of document $d$, $tf_{t,d}$ is the term frequency of term $t$ in document $d$, and $L_d$ is the number of tokens in document $d$. That is, we just count up how often each word occurred, and divide by the total number of words in the document $d$.

### 1.1.1. [15] Build TDM (or DTM)

The first thing we need to do is to build a term-document matrix for tour dataset.

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# TODO: build term-document matrix for the dataset
from nltk import word_tokenize
tdm = {}

# find all terms
for (id, doc) in enumerate(all_data):
    words = word_tokenize(doc.lower())
    for word in words:
      if word in tdm:
        if id in tdm[word]:
          tdm[word][id] += 1
        else:
          tdm[word][id] = 1
      else:
        tdm[word] = {}
        tdm[word][id] = 1

### 1.1.2. [30] Smoothing

Now, you need to implement the abovementioned logic in the `lm_rank_documents` function below. Do you see any potential problems?

Yes, data sparsity - we don't expect to meet each term in each doc, so, in most cases, we will get zero scores, which is not what we really want.

The solution is smooting.

One option is *[additive smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)* - adding a small number (0 to 1) to the observed counts and renormalizing to give a probability distribution.

Another option is called [Jelinek-Mercer smoothing](http://mlwiki.org/index.php/Smoothing_for_Language_Models#Jelinek-Mercer_Smoothing) - a simple idea that works well in practice is to use a mixture between a document-specific distribution and distribution estimated from the entire collection:

![](https://i.imgur.com/8Qv41Wp.png)

where 0 < λ < 1 and $M_c$ is a language model built from the entire document collection.

Refer to [*Chapter 12*](https://nlp.stanford.edu/IR-book/html/htmledition/language-models-for-information-retrieval-1.html) for the detailed explanation.


You are going to apply both in your `lm_rank_documents` function. This function takes TDM or DTM as an input, and ranks all documents "building" a language model for each document, returning relative probabilities of query being generated by a document as a document's score.

In [ ]:
import numpy as np

def lm_rank_documents(query, tdm, smoothing='additive', param=0.001):
    # TODO: score each document in tdm using this document's language model
    # implement two types of smoothing. Looks up term frequencies in tdm
    # return document scores in a convenient form
    # param is alpha for additive / lambda for jelinek-mercer
    # stores score for each document
    scores = np.zeros((15002, ))
    # in total how many different words
    d = len(tdm.keys())
    # how many words in total
    d_total = 0
    for freq in tdm.values():
      d_total += sum(freq.values())
    if smoothing == 'additive':
      for doc in range(15002):
        # how many words in current doc
        N = len(word_tokenize(all_data[doc].lower()))
        for q in query:
          q = q.lower()
          s = 1
          if q in tdm and doc in tdm[q]:
            s *= ((tdm[q][doc] + param) / (N + param * d))
          else:
            s *= (param / (N + param * d))
        scores[doc] = s
    if smoothing == 'jelinek-mercer':
      for doc in range(15002):
        # how many words in current doc
        N = len(word_tokenize(all_data[doc].lower()))
        a = 1
        b = 1
        for q in query:
          if q in tdm and doc in tdm[q]:
            a *= (tdm[q][doc] / N)
          else:
            a = 1e-7
          if q in tdm:  
            b *= (sum(tdm[q].values()) / d_total)
          else:
            b = 1e-7
        scores[doc] = (1 - param) * a + param * b
    return scores

### 1.1.3. [10] Testing

Check if this type of ranking gives meaningful results. For each query output document `category`, `doc_id`, `score`, and the *beginning* of the document, as it is shown below. Analyze if categories and contents match the queries. 

In [ ]:
def process_query(raw_query):
    # TODO: process user query and print search results including document category, id, score, and some part of it
    query = word_tokenize(raw_query)
    scores = lm_rank_documents(query, tdm)
    best_scores = np.argsort(scores)[::-1]
    print('user query: ', raw_query, '\n\nsearch results:\n')
    for id in best_scores[:5]:
      if id < 10000:
        print('braininjury', id, scores[id])
      elif id < 12073:
        print('economy', id, scores[id])
      elif id < 12918:
        print('fuel', id, scores[id])
      else:
        print('music', id, scores[id])
      print(all_data[id])

user_queries = ["piano concert", "symptoms of head trauma", "wall street journal"]
for q in user_queries:
    process_query(q)
    print("\n")

user query:  piano concert 

search results:

economy 10490 0.008563208918323618
atlanta prominent midtown intersection one step closer becoming major cultural landmark the atlanta symphony orchestra and houston developer announced six year master plan tuesday for its new symphony center and two residential towers peachtree and 14th streets the aso board also approved three key members design team for the two interlinked projects the master plan has undergone significant changes since was rolled out last spring the new plan relocates the concert hall off peachtree street giving the first condominium tower the prime corner real estate the high profile spot initially was intended for the new symphony hall but the project new master planner recommended that symphony center instead built 14th street between second condo tower and one atlantic center the two part project would help solidify midtown position the cultural heart metro atlanta would linked the woodruff arts center park offering

Sample results can look like this (if collapsed, click on 3 dots):

```
user query: piano concert

search results:
music 13330 0.012384164490679759
atlanta prominent midtown intersection one step closer becoming major cultural landmark the atlanta ...
economy 11335 0.012384164490679759
atlanta prominent midtown intersection one step closer becoming major cultural landmark the atlanta ...
music 12926 0.011382499792705511
felt like was going church marry guy never met said the jazz violinist regina carter the metaphorica...
music 14390 0.010661589922122
hailed los angeles brightest flower its flashiest ship sail its keenest architectural triumph perhap...
music 13818 0.010549141787975117
everything was finished sept the super bowl logo would reflection new orleans featuring streetcar an...


user query: symptoms of head trauma

search results:
brain_injury 7319 0.06022877378376099
the direct economic burden blunt and penetrating trauma managed care population background although ...
brain_injury 6987 0.05854539395767944
history reported head trauma sample women substance abuse treatment objectives determine the prevale...
brain_injury 5257 0.05760140208255336
violent head trauma china report cases background the occurrence violent trauma has recently increas...
brain_injury 1536 0.055365767080148634
mild head trauma and chronic headaches returning soldiers objective determine the incidence and type...
brain_injury 8874 0.05379997937839304
maxillofacial trauma major trauma patients background trauma has been identified major public health...


user query: wall street journal

search results:
economy 11294 0.027288833622119528
these business stories for release tuesday january are moving today clients the new york times news ...
economy 11295 0.027288833622119528
these business stories for release tuesday january are moving today clients the new york times news ...
music 14641 0.026716049665405375
these feature stories are moving today clients the new york times news service stories are for relea...
music 14640 0.026716049665405375
these feature stories are moving today clients the new york times news service stories are for relea...
economy 11297 0.025763725974814314
these feature stories are moving today clients the new york times news service stories are for relea...
```

## 1.2. [+10] Topic modeling

Now let's use *Latent Dirichlet Allocation* to identify topics in this collection and check if they match the original topics (fuel, economy, etc.). Go through the tutorial [here](https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0) and apply the ideas there to our dataset. 

In [ ]:
# TODO: apply LDA to our dataset and output the resulting categories 


We expect to see something like this (if collapsed, click on 3 dots):

```
Topic #0:
brain injury patients tbi traumatic study cerebral results severe group cognitive clinical pressure imaging following outcome control using children test

Topic #1:
new said york news atlanta like times year service time people undated just music journal constitution city says com years

Topic #2:
patients injury injuries trauma head study results traumatic brain treatment cases patient fractures years case outcome methods clinical tbi surgery

Topic #3:
said year bush percent new enron company president government people economy years million state companies states economic united time billion
```

# 2. [40+10] Sugges_

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest.

Today we will practice generating suggestions using [Trie](https://en.wikipedia.org/wiki/Trie) datastructure (prefix tree), see the example below.

Plan of your homework:

1. Build Trie based on real search query data, provided by AOL company;
2. Generate suggestion based on trie;
3. Measure suggestion speed;
4. (+) Optionally add spellcheck to suggest.


![image](https://www.ritambhara.in/wp-content/uploads/2017/05/Screen-Shot-2017-05-01-at-4.01.38-PM.png)

## 2.0. Install Trie data structure support

You are free to use any library implementation of Trie, as well as the one we suggest (read the docs before asking any questions!): https://github.com/google/pygtrie

In [4]:
!pip3 install pygtrie

### 2.0.1. Check it works and understand the example

In [5]:
import pygtrie
t = pygtrie.CharTrie()

# trie can be considered as a form of organizing a set of map
t["this is 1"] = "A"
t["this is 2"] = "B"
t["that is 3"] = "C"

print(t)

# "this" string is present in a set
n = t.has_node('this') == pygtrie.Trie.HAS_VALUE
# "this" prefix is present in a set
s = t.has_node('this') == pygtrie.Trie.HAS_SUBTRIE

print(f"Node = {n}\nSubtree = {s}")

# iterate a subtree
for key, val in t.iteritems("this"):
    print(key, '~', val)

CharTrie(this is 1: A, this is 2: B, that is 3: C)
Node = False
Subtree = True
this is 1 ~ A
this is 2 ~ B


## 2.1. Build a trie upon a dataset

### 2.1.1. [5] Read dataset

Download the [dataset](https://github.com/IUCVLab/information-retrieval/tree/main/datasets/aol) (we provide only the first part of the original data for simplicity (~3.5 mln queries)).

Explore the data, see readme file. Load the dataset.

In [3]:
!gunzip -d '/content/drive/MyDrive/user-ct-test-collection-01.txt.gz'

gzip: /content/drive/MyDrive/user-ct-test-collection-01.txt.gz: No such file or directory


In [1]:
import pandas as pd
#TODO: Read the dataset, e.g. as pandas dataframe

aol_data = pd.read_csv('/content/drive/MyDrive/user-ct-test-collection-01.txt', encoding='utf-8', sep='\t')

assert aol_data.shape[0] == 3558411, "Dataset size doesnt' match"

In [2]:
aol_data

,AnonID,Query,QueryTime,ItemRank,ClickURL
0,142,rentdirect.com,2006-03-01 07:17:12,NaN,NaN
1,142,www.prescriptionfortime.com,2006-03-12 12:31:06,NaN,NaN
2,142,staple.com,2006-03-17 21:19:29,NaN,NaN
3,142,staple.com,2006-03-17 21:19:45,NaN,NaN
4,142,www.newyorklawyersite.com,2006-03-18 08:02:58,NaN,NaN
...,...,...,...,...,...
3558406,24968114,-,2006-05-31 01:04:20,NaN,NaN
3558407,24969251,sp.trafficmarketplace.com,2006-05-31 15:51:23,NaN,NaN
3558408,24969374,orioles tickets,2006-05-31 12:24:51,NaN,NaN
3558409,24969374,orioles tickets,2006-05-31 12:31:57,2.0,http://www.greatseats.com


### 2.1.2. [10] Build Trie

We want suggest function to be **non-sensitive to stop words** because we don't want to upset the user if he confuses/omits prepositions, for example. Consider `public events in Innopolis` vs `public events at Innopolis` or `public events Innopolis` - they all mean the same.

Build Trie based on the dataset, **storing query statistics such as query frequency, urls and ranks in nodes**. Some queries may not have associated urls, others may have multiple ranked urls, consider this.

In [7]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import math
from tqdm import tqdm
nltk.download('punkt')

query_freq = {}
query_urls = {}

for index, data in tqdm(aol_data.iterrows(), total = len(aol_data)): 
  # update query frequency
  query = data.Query
  if query == '' or query == ' ' or pd.isnull(query):
    continue
  
  if query in query_freq:
    query_freq[query] += 1
  else:
    query_freq[query] = 1
  # update query urls
  url = data.ClickURL
  rank = data.ItemRank
  if not math.isnan(rank):
    if query in query_urls:
      query_urls[query].append((url, rank))
    else:
      query_urls[query] = [(url, rank)]


  0%|          | 0/3558411 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


100%|██████████| 3558411/3558411 [06:55<00:00, 8555.39it/s]


In [16]:
aol_trie = pygtrie.CharTrie(separator=' ')
stopWords = set('a on at of to is from for and with using the in &'.split(' '))

for Query, freq in tqdm(query_freq.items()):
  # tokenize query
  query = Query.lower().split(' ')
  # remove stop words
  filtered = []
  for q in query:
    if q not in stopWords:
      filtered.append(q)
  # check urls
  if Query in query_urls:
    urls = query_urls[Query]
  else:
    urls = []
  # update trie
  s = ' '.join(filtered)
  aol_trie[s] = (Query, freq, urls)

100%|██████████| 1216652/1216652 [02:09<00:00, 9374.43it/s] 


In [17]:
# test trie
bag = []
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)
    
    #NB: here we assume you store urls in a list field. But you can do something different. 
    bag += [url for url, rank in val[2]]
    
    assert "sample question" in key, "All examples have `sample question` substring"
    assert key[:len("sample question")] == "sample question", "All examples have `sample question` starting string"

for url in ["http://www.surveyconnect.com", "http://www.custominsight.com", 
            "http://jobsearchtech.about.com", "http://www.troy.k12.ny.us",
            "http://www.flinders.edu.au", "http://uscis.gov"]:
    assert url in bag, "This url should be in a try"

sample question surveys ~ ('sample question surveys', 5, [('http://www.surveyconnect.com', 7.0), ('http://www.custominsight.com', 4.0), ('http://www.askemployees.com', 10.0), ('http://www.lg-employers.gov.uk', 1.0)])
sample questions immigration interview ~ ('sample questions for immigration interview', 1, [])
sample questions interview ~ ('sample questions for interview', 1, [('http://www.quintcareers.com', 1.0)])
sample questions family interview ~ ('sample questions for family interview', 3, [('http://www.grandparents-day.com', 2.0), ('http://www.quintcareers.com', 5.0), ('http://jobsearchtech.about.com', 3.0)])
sample questions sociology race ethnicity ~ ('sample questions sociology race and ethnicity', 1, [])
sample questions biology ~ ('sample questions biology', 2, [('http://www.utexas.edu', 3.0), ('http://www.troy.k12.ny.us', 6.0)])
sample questions us citizenship test ~ ('sample questions for us citizenship test', 1, [('http://uscis.gov', 1.0)])
sample questionarie teaching ev

## 2.2. [10] Write a suggest function which is non-sensitive to stop words

Suggest options for user query based on Trie you just built.
Output results sorted by frequency, print query count for each suggestion. If there is an url available, print the url too. If multiple url-s are available, print the one with the highest rank (the less the better).

In [22]:
import numpy as np

def complete_user_query(query, trie, top_k=5):
    #TODO: suggest top_k options for a user query
    # sort results by frequency, suggest first ranked urls if available
    query = query.lower().split(' ')
    # remove stop words
    filtered = []
    for q in query:
      if q not in stopWords:
        filtered.append(q)
    query = ' '.join(filtered)
    queries = []
    freq = []
    urls = []
    try:
      for key, val in trie.iteritems(query):
        queries.append(val[0])
        freq.append(val[1])
        urls.append(val[2])
      freq = np.argsort(freq)[::-1]
      result = []
      for i in range(top_k):
        id = freq[i]
        result.append(queries[id])
        print('\tQuery:', queries[id])
        for url, rank in urls[id]:
          print('\t\tUrl:', url, 'Rank:', rank)
      return result
    except:
      return []
        
inp = "trie"
print("Query:", inp)
print("Results:")
res = complete_user_query(inp, aol_trie)

#NB we assume you return suggested query string only
assert res[0] == "tried and true tattoo"
assert res[1] == "triest" or res[1] == "triethanalomine"

Query: trie
Results:
	Query: tried and true tattoo
		Url: http://www.triedntruetattoo.com Rank: 1.0
		Url: http://www.triedntruetattoo.com Rank: 1.0
		Url: http://www.tattoonow.com Rank: 3.0
		Url: http://www.triedntruetattoo.com Rank: 1.0
		Url: http://www.triedntruetattoo.com Rank: 1.0
	Query: triethanalomine
		Url: http://avalon.unomaha.edu Rank: 1.0
		Url: http://www.mse.ncsu.edu Rank: 2.0
		Url: http://www.amazon.com Rank: 4.0
	Query: triest
	Query: tried and failed
	Query: triethanolamine
		Url: http://www.dermaxime.com Rank: 1.0


## 2.3. [10] Measure suggest speed ##

Check how fast your search is working. Consider changing your code if it takes too long on average.

In [23]:
import time

inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]
avg = 0
for query in inp_queries:
  print("Query:", query)
  print("Results:")
  start = time.time()
  res = complete_user_query(query, aol_trie)
  end = time.time()
  avg += (end - start)
  print("Time taken:", end-start)
print('Average time:', avg / len(inp_queries))

Query: inf
Results:
	Query: information clearing house
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www.informationclearinghouse.info Rank: 1.0
		Url: http://www

## 2.4. [5] Question
What is the empirical threshold for minimal prefix length for suggest query? Write your answer in a cell bellow and justify it (just few sentences).

% your answer here

## 2.5. [+10] Add spellchecking to your suggest

Try to make your search results as close as possible. Compare top-5 results of each query with top-5 results for corrected.

In [ ]:
inp_queries = ["inormation retrieval", "shelrock hol", "carnagie mell", "babis r", "Barrbara "]
inp_queries_corrected = ["information retrieval", "sherlock hol", "carnegie mell", "babies r", "Barbara "]


def complete_user_query_with_spellchecker(query, trie, top_k=5):
    #TODO: suggest top_k options for a user query
    # sort results by frequency, suggest first ranked urls if available
    pass


for q, qc in zip(inp_queries, inp_queries_corrected):
    assert  complete_user_query(qc, trie, 5) == \
            complete_user_query_with_spellchecker(q, trie, 5), "Assert {} and {} give different results".format(q, qc)